# **Import Libraries**

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,Input,Add,Activation,Concatenate
from tensorflow.keras import Model

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from tensorflow.keras.utils import plot_model
import scipy

from google.colab.drive import mount
mount("/content/drive")

Mounted at /content/drive


# **Read Data**

In [3]:
train_df=pd.read_csv("/content/drive/MyDrive/Data Sets/Horse/train.csv")
test_df=pd.read_csv("/content/drive/MyDrive/Data Sets/Horse/test.csv")
train_df.columns

Index(['id', 'surgery', 'age', 'hospital_number', 'rectal_temp', 'pulse',
       'respiratory_rate', 'temp_of_extremities', 'peripheral_pulse',
       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'nasogastric_reflux_ph', 'rectal_exam_feces', 'abdomen',
       'packed_cell_volume', 'total_protein', 'abdomo_appearance',
       'abdomo_protein', 'surgical_lesion', 'lesion_1', 'lesion_2', 'lesion_3',
       'cp_data', 'outcome'],
      dtype='object')

In [4]:
train_df.describe()

,id,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,1235.0000,1.235000e+03,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000
mean,617.0000,9.545004e+05,38.202186,79.574089,30.054251,4.382591,49.602429,21.388016,3.290931,3832.496356,14.612146,3.577328
std,356.6581,1.356403e+06,0.788668,29.108638,16.452066,1.937357,10.535800,26.676453,1.589195,5436.733774,193.705735,88.858953
min,0.0000,5.213990e+05,35.400000,30.000000,8.000000,1.000000,23.000000,3.500000,0.100000,0.000000,0.000000,0.000000
25%,308.5000,5.288000e+05,37.800000,53.000000,18.000000,2.000000,43.000000,6.600000,2.000000,2205.000000,0.000000,0.000000
50%,617.0000,5.297770e+05,38.200000,76.000000,28.000000,4.500000,48.000000,7.500000,3.000000,2209.000000,0.000000,0.000000
75%,925.5000,5.341450e+05,38.600000,100.000000,36.000000,6.000000,57.000000,9.100000,4.300000,3205.000000,0.000000,0.000000
max,1234.0000,5.305129e+06,40.800000,184.000000,96.000000,7.500000,75.000000,89.000000,10.100000,41110.000000,3112.000000,2209.000000


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1235 non-null   int64  
 1   surgery                1235 non-null   object 
 2   age                    1235 non-null   object 
 3   hospital_number        1235 non-null   int64  
 4   rectal_temp            1235 non-null   float64
 5   pulse                  1235 non-null   float64
 6   respiratory_rate       1235 non-null   float64
 7   temp_of_extremities    1235 non-null   object 
 8   peripheral_pulse       1235 non-null   object 
 9   mucous_membrane        1235 non-null   object 
 10  capillary_refill_time  1235 non-null   object 
 11  pain                   1235 non-null   object 
 12  peristalsis            1235 non-null   object 
 13  abdominal_distention   1235 non-null   object 
 14  nasogastric_tube       1235 non-null   object 
 15  naso

# **Visualize Data**

In [ ]:
ProfileReport(train_df)

# **Preprocessing Data**

In [6]:
train_df.columns

Index(['id', 'surgery', 'age', 'hospital_number', 'rectal_temp', 'pulse',
       'respiratory_rate', 'temp_of_extremities', 'peripheral_pulse',
       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'nasogastric_reflux_ph', 'rectal_exam_feces', 'abdomen',
       'packed_cell_volume', 'total_protein', 'abdomo_appearance',
       'abdomo_protein', 'surgical_lesion', 'lesion_1', 'lesion_2', 'lesion_3',
       'cp_data', 'outcome'],
      dtype='object')

In [7]:
train_df.drop(["id","hospital_number"],axis=1,inplace=True)

In [12]:
numericalVariables=[ 'rectal_temp', 'pulse','respiratory_rate',"pulse",'respiratory_rate', "nasogastric_reflux_ph",'packed_cell_volume', 'total_protein','abdomo_protein',"lesion_1"]

categoricalVariables=["abdomo_appearance","pain","capillary_refill_time","mucous_membrane","abdomen","rectal_exam_feces","nasogastric_reflux","nasogastric_tube","abdominal_distention","peristalsis","pain",'temp_of_extremities', 'peripheral_pulse',]

y_vector=pd.get_dummies(train_df["outcome"])

In [14]:
for i in categoricalVariables:
    print("--x--","\n")
    print(i,pd.unique(train_df[i]))
    print("--x--","\n")


--x-- 

abdomo_appearance ['serosanguious' 'cloudy' 'clear' 'None']
--x-- 

--x-- 

pain ['depressed' 'mild_pain' 'extreme_pain' 'alert' 'severe_pain' 'None'
 'slight']
--x-- 

--x-- 

capillary_refill_time ['more_3_sec' 'less_3_sec' 'None' '3']
--x-- 

--x-- 

mucous_membrane ['dark_cyanotic' 'pale_cyanotic' 'pale_pink' 'normal_pink' 'bright_pink'
 'bright_red' 'None']
--x-- 

--x-- 

abdomen ['distend_small' 'distend_large' 'normal' 'firm' 'None' 'other']
--x-- 

--x-- 

rectal_exam_feces ['decreased' 'absent' 'None' 'normal' 'increased' 'serosanguious']
--x-- 

--x-- 

nasogastric_reflux ['less_1_liter' 'more_1_liter' 'none' 'None' 'slight']
--x-- 

--x-- 

nasogastric_tube ['slight' 'none' 'significant' 'None']
--x-- 

--x-- 

abdominal_distention ['slight' 'moderate' 'none' 'severe' 'None']
--x-- 

--x-- 

peristalsis ['absent' 'hypomotile' 'normal' 'hypermotile' 'None' 'distend_small']
--x-- 

--x-- 

pain ['depressed' 'mild_pain' 'extreme_pain' 'alert' 'severe_pain' 'None'
 'sli

In [ ]:
train_df["cp_data"].replace({
    "yes":1,
    "no":0}
,inplace=True)

train_df["surgical_lesion"].replace({
    "yes":1,
    "no":0}
,inplace=True)

train_df["surgery"].replace({
    "yes":1,
    "no":0}
,inplace=True)

train_df["age"].replace({
    "adult":1,
    "young":0}
,inplace=True)

# **Visualize Data - 2**

In [ ]:
plt.suptitle("Box Plot for All Numerical Variables")
for i in numericalVariables:
    fig = px.box(train_df, y=i)
    fig.show()

# **Data Preprocessing - 2**

In [ ]:
numericalPipeline=Pipeline([

    ("Imputer",SimpleImputer(strategy="median")),
    ("StandardScaler",StandardScaler())

])

categoricalPipeline=Pipeline([

    ("Imputer",SimpleImputer(strategy="most_frequent")),
    ("OneHotEncoder",OneHotEncoder())

])

In [ ]:
Transformer=ColumnTransformer([

    ("NumericalVariables",numericalPipeline,numericalVariables),
    ("CategoricalVariables",categoricalPipeline,categoricalVariables)

])

In [ ]:
x_train_vector=Transformer.fit_transform(train_df)

In [ ]:
x_train_vector=scipy.sparse.lil_matrix(x_train_vector).toarray()


# **Model Building**

In [ ]:
inputlayer=Input((81,),name="inputlayer")

#Parallel Branch One
dense1=Dense(2056,activation="relu")(inputlayer)
dense2=Dense(1024,activation="relu")(dense1)
dense3=Dense(512,activation="relu")(dense2)
dense4=Dense(256,activation="relu")(dense3)
dense5=Dense(256,activation="relu")(dense4)

#Parallel Branch One
dense6=Dense(2056,activation="tanh")(inputlayer)
dense7=Dense(1024,activation="relu")(dense6)
dense8=Dense(512,activation="relu")(dense7)
dense9=Dense(256,activation="relu")(dense8)
dense10=Dense(256,activation="relu")(dense9)

#Concatenate Different Outputs
add=Activation("relu")(Add()([dense5,dense10]))



dense11=Dense(1024,activation="relu")(add)
dense12=Dense(1024,activation="tanh")(dense11)
dense13=Dense(512,activation="relu")(dense12)
dense14=Dense(512,activation="relu")(dense13)
output=Dense(3,activation="softmax")(dense14)

In [ ]:
ml=Model(inputs=inputlayer,outputs=output)
plot_model(ml,show_layer_names=True,show_layer_activations=True,show_shapes=True)

In [ ]:
ml.compile(

    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"
)

In [ ]:
ml.fit(x=x_train_vector,y=y_vector,epochs=100,verbose=2)